In [1]:
%env SOFA_ROOT=/home/erik/sofa/build/install
%env SOFAPYTHON3_ROOT=/home/erik/sofa/build/install/install/plugins/SofaPython3

env: SOFA_ROOT=/home/erik/sofa/build/install
env: SOFAPYTHON3_ROOT=/home/erik/sofa/build/install/install/plugins/SofaPython3


In [2]:
import numpy as np
from imitation.algorithms import bc
from imitation.data.types import Trajectory
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.data.rollout import flatten_trajectories
from sofa_env.base import RenderMode
from sofa_env.scenes.ligating_loop.ligating_loop_env import LigatingLoopEnv, ObservationType, ActionType
from gymnasium.wrappers import TimeLimit
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv

---------------------------------------
Checking SOFA_ROOT and SOFAPYTHON3_ROOT
Using environment variable SOFA_ROOT: /home/erik/sofa/build/install
---------------------------------------


In [3]:
def _make_env():
    """Helper function to create a single environment. Put any logic here, but make sure to return a RolloutInfoWrapper."""
    _env = LigatingLoopEnv(
        observation_type=ObservationType.RGBD,
        render_mode=RenderMode.HUMAN,
        action_type=ActionType.CONTINUOUS,
        image_shape=(256, 256),
        frame_skip=1,
        time_step=0.1,
        settle_steps=50,
    )
    _env = TimeLimit(_env, max_episode_steps=500)
    _env = RolloutInfoWrapper(_env)
    return _env


def _npz_to_traj(n_traj: 500):
    ret = []
    for i in range(n_traj):
        path = f'/home/erik/sofa_env_demonstrations/ligating_loop/LigatingLoopEnv_{i}.npz'
        npz_data = np.load(path)
        print(np.append(npz_data['rgb'], npz_data['depth'][..., np.newaxis], 3).shape)
        ret.append(Trajectory(np.append(npz_data['rgb'], npz_data['depth'][..., np.newaxis], 3),
                              npz_data['action'], None, True))

    return ret

In [4]:
env = DummyVecEnv([_make_env for _ in range(1)])
traj = _npz_to_traj(1)  

AttributeError: 'LigatingLoopEnv' object has no attribute 'observation_type'

In [5]:
from stable_baselines3.common.policies import ActorCriticPolicy

transitions = flatten_trajectories(traj)
print(transitions.acts.shape, transitions.obs.shape)
rng = np.random.default_rng()
policy = ActorCriticPolicy(env.observation_space, env.action_space, lambda epoch: 1e-3 * 0.99 ** epoch )



(495, 5) (495, 256, 256, 4)


In [6]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    policy=policy,
    rng=rng,
    device='cuda'
)

In [7]:
#reward_before_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
#print(f"Reward before training: {reward_before_training}")

In [8]:
bc_trainer.train(n_epochs=1)

0batch [00:00, ?batch/s]


<class 'torch.Tensor'> cpu


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [13]:
reward_after_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"Reward after training: {reward_after_training}")

/home/erik/miniconda3/envs/sofa/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


[INFO]    [SofaPython3] Initializing with python version 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
[INFO]    [SofaPython3] Shared library name is 'libpython3.9m.so'
[INFO]    [SofaPython3] Added '/home/erik/sofa/build/install/plugins/SofaPython3/lib/python3/site-packages' to sys.path
[INFO]    [PluginManager] Loaded plugin: /home/erik/sofa/build/install/plugins/SofaPython3/lib/libSofaPython3.so
[INFO]    [PluginManager] Loaded plugin: /home/erik/sofa/build/install/lib/libSofa.Component.Mapping.Linear.so
[INFO]    [PluginManager] Loaded plugin: /home/erik/sofa/build/install/lib/libSofa.Component.Topology.Mapping.so
[INFO]    [PluginManager] Loaded plugin: /home/erik/sofa/build/install/lib/libSofa.Component.Constraint.Lagrangian.Model.so
[INFO]    [PluginManager] Loaded plugin: /home/erik/sofa/build/install/lib/libSofa.Component.Engine.Select.so
[INFO]    [PluginManager] Loaded plugin: /home/erik/sofa/build/install/lib/libSofa.Component.Constraint.Lagrangian.Correction.so
[INFO]

[ERROR]   [FastTetrahedralCorotationalForceField(FastTetrahedralCorotationalForceField)] TopologyData: edgeInfo already has a TopologyDataHandler. createTopologyHandler should only be called once at init of the TopologyData.
[ERROR]   [FastTetrahedralCorotationalForceField(FastTetrahedralCorotationalForceField)] TopologyData: pointInfo already has a TopologyDataHandler. createTopologyHandler should only be called once at init of the TopologyData.
[ERROR]   [FastTetrahedralCorotationalForceField(FastTetrahedralCorotationalForceField)] TopologyData: tetrahedronInfo already has a TopologyDataHandler. createTopologyHandler should only be called once at init of the TopologyData.
[ERROR]   [UniformMass(UniformMass)] TopologyData: indices already has a TopologyDataHandler. createTopologyHandler should only be called once at init of the TopologyData.
[ERROR]   [FixedConstraint(FixedConstraint)] TopologyData: indices already has a TopologyDataHandler. createTopologyHandler should only be called

[WARNING] [FastTetrahedralCorotationalForceField(FastTetrahedralCorotationalForceField)] buildStiffnessMatrix not implemented: for compatibility reason, the deprecated API (addKToMatrix) will be used. This compatibility will disapear in the future, and will cause issues in simulations. Please update the code of FastTetrahedralCorotationalForceField to ensure right behavior: the function addKToMatrix has been replaced by buildStiffnessMatrix
[WARNING] [FastTetrahedralCorotationalForceField(FastTetrahedralCorotationalForceField)] buildDampingMatrix not implemented: for compatibility reason, the deprecated API (addBToMatrix) will be used. This compatibility will disapear in the future, and will cause issues in simulations. Please update the code of FastTetrahedralCorotationalForceField to ensure right behavior: the function addBToMatrix has been replaced by buildDampingMatrix
[WARNING] [UniformVelocityDampingForceField(UniformVelocityDampingForceField)] buildStiffnessMatrix not implemente